In [1]:
import torch

from data.prompt_dataset import PromptedEhvoy
from data.ehovy_race import EhovyRaceDataset

from model.llama_3_2_tokenizer import Llama32Tokenizer
from model.llama_3_2 import Llama32
from model.options_picker import OptionsPicker

/home/ubuntu/miniconda3/envs/main/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = EhovyRaceDataset(variation="high", split="train", max_article_size=800)
data_prompted = PromptedEhvoy(data)

Filtrando artículos en train: 100%|██████████| 62445/62445 [00:01<00:00, 48782.37 examples/s]


In [3]:
data_prompted[0]

('Context: The air hostess   was in a small kitchen at the back of the plane, preparing the plates for lunch, when a little old lady came and spoke to her, "Could you please tell me," she asked, "where is the ladies\' lavatory   in the plane?"\n"Yes, madam," said the air hostess and smiled. "It is right at the other end of the plane---at the front."\nThe little lady went too far. She walked all the way to the front of the plane, opened the door in front of her, and saw the captain of the plane and the other officers. They were all busy with their work and did not see her. She went out again, shut the door and returned to the air hostess.\n"Oh, didn\'t you find it, madam?" the girl asked her. "Yes, I did," said the little lady. "But there are four men in the ladies\' lavatory watching television."\n\nQuestion: The story happened  _  .\n\nOptions:\nA) in the evening\nB) in the afternoon\nC) in the morning\nD) at midnight\n\nAnswer:',
 'C')

In [12]:
model_name = "meta-llama/Llama-3.2-1B"

tokenizer = Llama32Tokenizer(model_name)
model = Llama32(model_name, do_sample=False)
options = [" A", " B", " C", " D"]
options_picker = OptionsPicker(model, tokenizer, options=options, device="cuda")

In [13]:
x, y = data_prompted[0]
x

'Context: The air hostess   was in a small kitchen at the back of the plane, preparing the plates for lunch, when a little old lady came and spoke to her, "Could you please tell me," she asked, "where is the ladies\' lavatory   in the plane?"\n"Yes, madam," said the air hostess and smiled. "It is right at the other end of the plane---at the front."\nThe little lady went too far. She walked all the way to the front of the plane, opened the door in front of her, and saw the captain of the plane and the other officers. They were all busy with their work and did not see her. She went out again, shut the door and returned to the air hostess.\n"Oh, didn\'t you find it, madam?" the girl asked her. "Yes, I did," said the little lady. "But there are four men in the ladies\' lavatory watching television."\n\nQuestion: The story happened  _  .\n\nOptions:\nA) in the evening\nB) in the afternoon\nC) in the morning\nD) at midnight\n\nAnswer:'

In [14]:
x_tokenized = tokenizer(x, padding="max_length", truncation=True, return_tensors="pt", max_length=900)

In [15]:
out = options_picker(x_tokenized["input_ids"], x_tokenized["attention_mask"])

In [16]:
print(out)

tensor([2.3601e-07, 1.4969e-06, 2.1230e-05, 9.3522e-06])


In [17]:
answer = options[torch.argmax(out)]
answer

' C'

In [18]:
# Let's see how the model performs on the first 100 examples

In [19]:
correct_predictions = 0
validation_length = len(data_prompted)

for i in range(validation_length):
    x, y = data_prompted[i]
    x = x.replace("\n\nAnswer:", "\n\nAfter analyze that, the option i choose between (A, B, C or D) is ")
    x = f"""
     You are a smart question answering model. Answer the question based on the next information, and at the end
     you will find the answer options. Choose the best one, only give the letter of the answer which could be A, B, C or D.
     {x}"""
    x_tokenized = tokenizer(x, padding="max_length", truncation=True, return_tensors="pt", max_length=512)
    out = options_picker(x_tokenized["input_ids"], x_tokenized["attention_mask"])
    answer = options[torch.argmax(out)]
    if answer == y:
        correct_predictions += 1

    if i % 10 == 0:
        print(out)
        print(x)
        print(answer, y)
        print(f"Processed {i} examples, current accuracy: {correct_predictions / (i + 1):.2f}")

accuracy = correct_predictions / validation_length
accuracy

tensor([6.8912e-08, 4.3698e-07, 6.3998e-06, 1.9014e-06])

     You are a smart question answering model. Answer the question based on the next information, and at the end
     you will find the answer options. Choose the best one, only give the letter of the answer which could be A, B, C or D.
     Context: The air hostess   was in a small kitchen at the back of the plane, preparing the plates for lunch, when a little old lady came and spoke to her, "Could you please tell me," she asked, "where is the ladies' lavatory   in the plane?"
"Yes, madam," said the air hostess and smiled. "It is right at the other end of the plane---at the front."
The little lady went too far. She walked all the way to the front of the plane, opened the door in front of her, and saw the captain of the plane and the other officers. They were all busy with their work and did not see her. She went out again, shut the door and returned to the air hostess.
"Oh, didn't you find it, madam?" the girl asked her. "Yes, 

KeyboardInterrupt: 

In [36]:
x, y = data_prompted[90]



In [37]:
x = f"""You are a smart question answering model. Answer the question based on the next information, and at the end you should pick the right answer. Choose the best one, only give the letter of the answer which could be A, B, C or D. You must give your answer in a single letter.
     {x}"""
x, y

("You are a smart question answering model. Answer the question based on the next information, and at the end you should pick the right answer. Choose the best one, only give the letter of the answer which could be A, B, C or D. You must give your answer in a single letter.\n     Context: Part-time Waitress\nBusy cafe needs honest and good-looking waitress for weekends. Must enjoy working with a team and dealing with customers. Call Li Ling at 0732-8536724 after 6 pm.\nDelivery   Person\nYoung, healthy person able to deliver heavy boxes of books. Must have a driver's license and can carry heavy boxes. We're looking for a person for this position. Call Liu Fang at 0732-7887766 at any time.\n _ Wanted\nKind, hard-working nanny wanted to look after three friendly children. Must be experienced and have childcare quail fications . Please call Chen Yiping at 0732-6774538 between 10 a.m. and 4 p.m\n\nQuestion: These are   _   in the newspaper.\n\nOptions:\nA) notices\nB) ads\nC) stories\nD) n

In [38]:
x_tokenized = tokenizer(x, padding="max_length", truncation=True, return_tensors="pt", max_length=512)
generation = model.model.generate(
    input_ids=x_tokenized["input_ids"],
    attention_mask=x_tokenized["attention_mask"],
    max_length=1024,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [39]:
out = tokenizer.decode(generation[0])

In [40]:
out

"You are a smart question answering model. Answer the question based on the next information, and at the end you should pick the right answer. Choose the best one, only give the letter of the answer which could be A, B, C or D. You must give your answer in a single letter.\n     Context: Part-time Waitress\nBusy cafe needs honest and good-looking waitress for weekends. Must enjoy working with a team and dealing with customers. Call Li Ling at 0732-8536724 after 6 pm.\nDelivery   Person\nYoung, healthy person able to deliver heavy boxes of books. Must have a driver's license and can carry heavy boxes. We're looking for a person for this position. Call Liu Fang at 0732-7887766 at any time.\n _ Wanted\nKind, hard-working nanny wanted to look after three friendly children. Must be experienced and have childcare quail fications. Please call Chen Yiping at 0732-6774538 between 10 a.m. and 4 p.m\n\nQuestion: These are   _   in the newspaper.\n\nOptions:\nA) notices\nB) ads\nC) stories\nD) new